This method uses the Okubo–Weiss parameter following the method described in:
[1] https://link.springer.com/content/pdf/10.1007/s10236-013-0680-7.pdf

Limitation:
- noise in the W field
- detects an excess of eddies (Sadarjoenand Post 2000; Chaigneau et al. 2008)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib notebook

Read data

In [5]:
from scipy.io import netcdf_file

# https://resources.marine.copernicus.eu/product-download/SEALEVEL_EUR_PHY_L4_MY_008_068
#dataset = 'data10102019'
dataset = "01052014"

f = netcdf_file(dataset+".nc")
lat = f.variables['latitude'].data
long = f.variables['longitude'].data
vel_x = f.variables['ugos'].data.squeeze()
vel_y = f.variables['vgos'].data.squeeze()


# fill in missing values
from common_functions import interpolate_missing_point

#imputed_vel_x = interpolate_missing_point(vel_x, np.ma.masked_invalid(vel_x).mask)
#imputed_vel_y = interpolate_missing_point(vel_y, np.ma.masked_invalid(vel_y).mask)
imputed_vel_x = np.ma.masked_where(np.isnan(vel_x), vel_x)
imputed_vel_y = np.ma.masked_where(np.isnan(vel_y), vel_y)

## Compute the Okubo-Weiss parameter

In [6]:
from common_functions import compute_okubo_weiss_parameter

def OW(r):
    W = compute_okubo_weiss_parameter(imputed_vel_x, imputed_vel_y)
    W_0 = -0.2 * np.std(W)
    W_eddies = np.ma.where(W < W_0, W, 0)
    from scipy import ndimage
    W_eddies = ndimage.maximum_filter(W_eddies, size=(r,r))
    W_eddies_ma = np.ma.masked_where(W_eddies == 0, np.hypot(imputed_vel_x, imputed_vel_y))
    from scipy.ndimage import label
    labels, numL = label(W_eddies)
    eddies = []
    for i in range(1, numL+1):
        eddies.append(np.mean(np.argwhere(labels == i),axis=0))
    eddies = np.asarray(eddies)
    from scipy.interpolate import RegularGridInterpolator

    m,n = imputed_vel_x.shape
    i = np.linspace(0, m, m)
    j = np.linspace(0, n, n)
    method = "linear"

    latm, longm = np.meshgrid(lat, long, indexing='ij')

    interp_lat = RegularGridInterpolator((i, j), latm, method=method)
    interp_long = RegularGridInterpolator((i, j), longm, method=method)

    return np.column_stack([interp_lat(eddies), interp_long(eddies)])

In [41]:
from dataclasses import dataclass

from scipy.spatial import KDTree
from sklearn.metrics.pairwise import haversine_distances

real_data = pd.read_csv("validation_data/01052014.csv")
real_data = real_data[["Lat","Lon"]].to_numpy()
real_data = real_data[(real_data[:,1] > -30) & (real_data[:,1] < -20) & (real_data[:,0] > 20) & (real_data[:,0] < 60)]

@dataclass
class Method:
    name: str
    points: np.ndarray
    P: int = 0
    TP: int = 0
    FP: int = 0
        
    def compare(self, ground_truth: np.ndarray, dist_tol):
        '''dist_tol is in km'''
        self.P = ground_truth.shape[0]
        
        tree_detected = KDTree(self.points)
        dist, index = tree_detected.query(ground_truth[:,0:2])
        truth = np.deg2rad(ground_truth[:,0:2])
        detected = np.deg2rad(self.points[index,:])
        dist_mat = haversine_distances(truth, detected)
        dist = np.diagonal(dist_mat) * 6371000/1000  # multiply by Earth radius to get kilometers
        
        self.TP = (dist < dist_tol).sum()
        self.mean_error = np.mean(dist[dist < dist_tol])
        self.FN = self.P - self.TP
        
        tree_truth = KDTree(ground_truth[:,0:2])
        dist, index = tree_truth.query(self.points)
        detected = np.deg2rad(ground_truth[index,0:2])
        dist_mat = haversine_distances(truth, detected)
        dist = np.diagonal(dist_mat) * 6371000/1000  # multiply by Earth radius to get kilometers
                
        self.FP = (dist > dist_tol).sum()
    
    @property
    def TPR(self):
        return self.TP/self.P
    
    @property
    def FDR(self):
        return self.FP/(self.FP + self.TP)
    
    @property
    def F1(self):
        return self.TP/(self.TP + 0.5*(self.FP + self.FN))

In [42]:
res = []
for r in range(0,10):
    try:
        method = Method("",OW(r))
        method.compare(real_data, 50)
        res.append(method.FDR)
    except:
        res.append(0)

/home/jakeb/development/mdm-phase-a-data/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jakeb/development/mdm-phase-a-data/venv/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [43]:
res

[0.5275590551181102,
 0.5275590551181102,
 0.5541666666666667,
 0.6273584905660378,
 0.7326732673267327,
 0.7058823529411765,
 1.0,
 0,
 0,
 0]